In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=ee38ec6925a9e2e153e10b0fed61c76944d71f1387a4a7a8f3ce26cbc0f54771
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("TransformOpenAQData").config("spark.jars", "/content/postgresql-42.6.0.jar").getOrCreate()

# 0) Test database connection with pyspark

In [3]:
url = "jdbc:postgresql://b9t5hul4bx.ff2skdqvke.tsdb.cloud.timescale.com:34391/tsdb"

properties = {
    "user": "tsdbadmin",
    "password": "d9x6um9xy1sv5h6w",
    "driver": "org.postgresql.Driver"
}

In [5]:
table_name = "airpollution"
query = f"(SELECT * FROM {table_name} LIMIT 1) AS subquery"
df = spark.read.jdbc(url=url, table=query, properties=properties)
df.show()

+---+-------+----+---------+--------+----------+---------------+
| id|country|city|parameter|utc_time|local_time|rolling_24h_avg|
+---+-------+----+---------+--------+----------+---------------+
+---+-------+----+---------+--------+----------+---------------+



### Everything is normal, db is empty

# 1) Read Data

In [8]:
ndjson_file_path = "datasets/*.ndjson"

# Read the NDJSON file into a DataFrame
airpollution = spark.read.json(ndjson_file_path)

# 2) Data Exploration

In [9]:
airpollution.printSchema()

root
 |-- attribution: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |-- averagingPeriod: struct (nullable = true)
 |    |-- unit: string (nullable = true)
 |    |-- value: double (nullable = true)
 |-- city: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- date: struct (nullable = true)
 |    |-- local: string (nullable = true)
 |    |-- utc: string (nullable = true)
 |-- location: string (nullable = true)
 |-- mobile: boolean (nullable = true)
 |-- parameter: string (nullable = true)
 |-- sourceName: string (nullable = true)
 |-- sourceType: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- value: double (nullable = true)



In [10]:
airpollution.head(2)

[Row(attribution=[Row(name='EPA AirNow DOS', url='http://airnow.gov/index.cfm?action=airnow.global_summary')], averagingPeriod=Row(unit='hours', value=1.0), city='Kabul', coordinates=Row(latitude=34.535812, longitude=69.190514), country='AF', date=Row(local='2021-08-15T20:00:00+04:30', utc='2021-08-15T15:30:00.000Z'), location='US Diplomatic Post: Kabul', mobile=False, parameter='pm25', sourceName='StateAir_Kabul', sourceType='government', unit='µg/m³', value=-999.0),
 Row(attribution=[Row(name='EPA AirNow DOS', url='http://airnow.gov/index.cfm?action=airnow.global_summary')], averagingPeriod=Row(unit='hours', value=1.0), city='Kabul', coordinates=Row(latitude=34.535812, longitude=69.190514), country='AF', date=Row(local='2021-08-15T21:00:00+04:30', utc='2021-08-15T16:30:00.000Z'), location='US Diplomatic Post: Kabul', mobile=False, parameter='pm25', sourceName='StateAir_Kabul', sourceType='government', unit='µg/m³', value=-999.0)]

In [11]:
airpollution.tail(2)

[Row(attribution=[Row(name='EEA', url='http://www.eea.europa.eu/themes/air/air-quality')], averagingPeriod=Row(unit='hours', value=1.0), city='Latina', coordinates=Row(latitude=41.531389, longitude=12.946111), country='IT', date=Row(local='2021-10-06T16:00:00+02:00', utc='2021-10-06T14:00:00.000Z'), location='IT1184A', mobile=False, parameter='no2', sourceName='EEA Italy', sourceType='government', unit='µg/m³', value=23.0),
 Row(attribution=[Row(name='EEA', url='http://www.eea.europa.eu/themes/air/air-quality')], averagingPeriod=Row(unit='hours', value=1.0), city='Napoli', coordinates=Row(latitude=40.85361, longitude=14.27167), country='IT', date=Row(local='2021-10-06T16:00:00+02:00', utc='2021-10-06T14:00:00.000Z'), location='IT1491A', mobile=False, parameter='no2', sourceName='EEA Italy', sourceType='government', unit='µg/m³', value=43.41)]

In [12]:
row_count = airpollution.count()
print("Number of rows:", row_count)

Number of rows: 77273


In [13]:
num_columns = len(airpollution.columns)
print("Number of columns:", num_columns)

Number of columns: 13


In [14]:
column_names = airpollution.columns
print("Column names:", column_names)

Column names: ['attribution', 'averagingPeriod', 'city', 'coordinates', 'country', 'date', 'location', 'mobile', 'parameter', 'sourceName', 'sourceType', 'unit', 'value']


In [15]:
print("Column data types:")
airpollution.dtypes

Column data types:


[('attribution', 'array<struct<name:string,url:string>>'),
 ('averagingPeriod', 'struct<unit:string,value:double>'),
 ('city', 'string'),
 ('coordinates', 'struct<latitude:double,longitude:double>'),
 ('country', 'string'),
 ('date', 'struct<local:string,utc:string>'),
 ('location', 'string'),
 ('mobile', 'boolean'),
 ('parameter', 'string'),
 ('sourceName', 'string'),
 ('sourceType', 'string'),
 ('unit', 'string'),
 ('value', 'double')]

# 3) Filter the Data
## Filter some countries like Great Britain (GB), France (FR), Netherlands(NL)
## Filter some columns

In [16]:
selected_columns = ["country", "city", "date", "parameter", "value", "averagingPeriod", "mobile", "unit"]
condition1 = (airpollution["country"] == "GB")
condition2 = (airpollution["country"] == "FR")
condition3 = (airpollution["country"] == "NL")
combined_condition = condition1 | condition2 | condition3

In [17]:
airpollution_some = airpollution.filter(combined_condition).select(selected_columns)

In [18]:
row_count = airpollution_some.count()
print("Number of rows:", row_count)

Number of rows: 22959


In [19]:
airpollution_some.printSchema()

root
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- date: struct (nullable = true)
 |    |-- local: string (nullable = true)
 |    |-- utc: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: double (nullable = true)
 |-- averagingPeriod: struct (nullable = true)
 |    |-- unit: string (nullable = true)
 |    |-- value: double (nullable = true)
 |-- mobile: boolean (nullable = true)
 |-- unit: string (nullable = true)



# 4) Data Manipulation

In [20]:
from pyspark.sql.functions import col, count

In [21]:
airpollution_some = airpollution_some.withColumn("utc_time", col("date.utc"))
airpollution_some = airpollution_some.withColumn("local_time", col("date.local"))
airpollution_some = airpollution_some.drop("date")

In [22]:
airpollution_some = airpollution_some.withColumn("time_unit", col("averagingPeriod.unit"))
airpollution_some = airpollution_some.withColumn("time_value", col("averagingPeriod.value"))
airpollution_some = airpollution_some.drop("averagingPeriod")

Before grouping data by country, city and parameter, we need to check if there are more distinct values than 1 for;  
1.   **time_unit**
2.   **time_value**
3.   **mobile**
4.   **unit**

If there are more options, we couldn't make a group from different type of units while computing the 24h average.

### 1. time_unit and 2. time_value

In [23]:
time_unit_values = airpollution_some.groupBy("country", "city", "parameter", "time_unit").agg(count("time_unit").alias("time_unit"))
print("Distinct time_unit Values")
time_unit_values.show()

Distinct time_unit Values
+-------+----------------+---------+---------+---------+
|country|            city|parameter|time_unit|time_unit|
+-------+----------------+---------+---------+---------+
|     GB|    Fort William|      no2|    hours|        5|
|     GB|          Bexley|      so2|    hours|       80|
|     FR|   Pas-de-Calais|      so2|    hours|       10|
|     FR|    Haute-Savoie|     pm10|    hours|       62|
|     FR|   French Guiana|       o3|    hours|       21|
|     GB|         Hackney|       o3|    hours|       76|
|     NL|       Hoogvliet|       o3|    hours|        2|
|     GB|      Eastbourne|     pm25|    hours|       81|
|     GB|          Harrow|      no2|    hours|      155|
|     FR|             Ain|      no2|    hours|       22|
|     FR|           Marne|     pm25|    hours|       12|
|     GB|      Sunderland|     pm10|    hours|        5|
|     GB|      Manchester|     pm10|    hours|       10|
|     GB|         Bromley|     pm25|    hours|       35|
|    

In [24]:
time_value_values = airpollution_some.groupBy("country", "city", "parameter", "time_value").agg(count("time_value").alias("time_value"))
print("Distinct time_value Values")
time_value_values.show()

Distinct time_value Values
+-------+--------------------+---------+----------+----------+
|country|                city|parameter|time_value|time_value|
+-------+--------------------+---------+----------+----------+
|     GB|              London|     pm10|      24.0|        55|
|     GB|           Ballymena|      no2|       1.0|         5|
|     FR|            Bas-Rhin|      no2|       1.0|        49|
|     FR|               Somme|     pm10|       1.0|        14|
|     NL|            Eibergen|      no2|       1.0|         2|
|     GB|             Norwich|     pm25|      24.0|         5|
|     GB|             Chatham|     pm10|      24.0|         5|
|     GB|         Port Talbot|     pm10|      24.0|         5|
|     FR|               Isère|      no2|       1.0|       120|
|     FR|Pyrénées-Atlantiques|     pm10|       1.0|        31|
|     FR|                 Ain|     pm25|       1.0|        10|
|     GB|            Lewisham|      no2|       1.0|       294|
|     FR|       Haute-Garonn

There is no different type of **time_unit** and **time_value** in the same city we can **drop** them from the dataframe.

In [25]:
airpollution_some = airpollution_some.drop("time_unit")
airpollution_some = airpollution_some.drop("time_value")

### 3. mobile

In [26]:
mobile_values = airpollution_some.groupBy("country", "city", "parameter", "mobile").agg(count("mobile").alias("mobile"))
print("Distinct mobile Values")
mobile_values.show()

Distinct mobile Values
+-------+----------------+---------+------+------+
|country|            city|parameter|mobile|mobile|
+-------+----------------+---------+------+------+
|     GB|Stanford-le-Hope|     pm25| false|     5|
|     GB|       Liverpool|      so2| false|     5|
|     GB|      Scunthorpe|      so2| false|     5|
|     GB|          Ealing|     pm10| false|   298|
|     GB|          Harrow|     pm10| false|   150|
|     NL|          Cabauw|     pm10| false|     2|
|     GB|          London|     pm25| false|    45|
|     GB|         Chatham|     pm10| false|     5|
|     GB|         Lambeth|     pm10| false|   223|
|     FR|           Meuse|       o3| false|     8|
|     GB|   West Midlands|      no2| false|    10|
|     GB|        Richmond|     pm10| false|    98|
|     FR|   Pas-de-Calais|      so2| false|    10|
|     FR|         Essonne|      no2| false|    34|
|     FR|     Haute-Corse|     pm10| false|     6|
|     NL|         Utrecht|      no2| false|     6|
|     NL

There is **no mobile 'true version'** in the same city we can drop it from the dataframe.

In [27]:
airpollution_some = airpollution_some.drop("mobile")

### 4. unit

In [28]:
unit_values = airpollution_some.groupBy("country", "city", "parameter", "unit").agg(count("unit").alias("unit"))
print("Distinct unit Values")
unit_values.show()

Distinct unit Values
+-------+--------------------+---------+-----+----+
|country|                city|parameter| unit|unit|
+-------+--------------------+---------+-----+----+
|     FR|        Haute-Vienne|     pm10|µg/m³|  17|
|     NL|           Rotterdam|     pm10|µg/m³|   4|
|     GB|           Sheffield|       o3|µg/m³|   5|
|     NL|           Oude Meer|      no2|µg/m³|   2|
|     GB|             Wrexham|      no2|µg/m³|   5|
|     FR|       Pas-de-Calais|     pm25|µg/m³|  28|
|     FR|               Yonne|       o3|µg/m³|  28|
|     FR|            Calvados|     pm10|µg/m³|  18|
|     GB|           Edinburgh|     pm10|µg/m³|   5|
|     GB|           Liverpool|       o3|µg/m³|  10|
|     GB|Barking and Dagenham|      so2|µg/m³|  93|
|     GB|           Greenwich|      no2|µg/m³| 671|
|     GB|       Tower Hamlets|     pm25|µg/m³|   8|
|     FR|           Haut-Rhin|      so2|µg/m³|  20|
|     FR|       Haute-Garonne|       o3|µg/m³|  40|
|     FR|               Loire|     pm10|µg/

There is **no different type of unit** in the same city we can drop it from the dataframe.

In [29]:
airpollution_some = airpollution_some.drop("unit")

## Parameter Values and Data Modelling
Parameter is the important key in the data because we compute the 24h average of the parameters' values. I will decide how to design data in the database according to relations among parameters.

In [30]:
parameter_values = airpollution_some.groupBy("parameter").agg(count("parameter").alias("parameter"))
print("Distinct Parameter Values")
parameter_values.show()

Distinct Parameter Values
+---------+---------+
|parameter|parameter|
+---------+---------+
|      so2|      863|
|       co|      177|
|       o3|     3541|
|     pm10|     6180|
|      no2|     9303|
|     pm25|     2895|
+---------+---------+



I would like to merge data by creating a column for each different sensor. Afterwards, I can store different parameters' values in the same row. And so I can show the data with simpler queries in a dashboard without needing filters and joins. But line 50 tells us that different sensors (so2, co etc.) collect data in different datetime. If we merge different sensor data at the same datetime, our data will be sparse data. Finally sparse data will not be a good option interms of storage efficiency.
However we don't need to create a table for different sensor data. We will create a single table in the database for all different sensor data to fecth data with simpler queries.

### Final Data Schema

In [31]:
airpollution_some.printSchema()

root
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: double (nullable = true)
 |-- utc_time: string (nullable = true)
 |-- local_time: string (nullable = true)



# 5) Data Aggregation
### A. Compute the 24hr rolling average for the value over utc_time grouped by country and city.

In [32]:
from pyspark.sql.window import Window
from pyspark.sql.functions import avg

In [33]:
# Define the window specification for the 24-hour rolling average
window_spec = Window.partitionBy("country", "city", "parameter").orderBy("utc_time").rowsBetween(-23, 0)

In [34]:
# Compute the 24-hour rolling average
airpollution_some = airpollution_some.withColumn("rolling_24h_avg", avg(col("value")).over(window_spec))

In [48]:
airpollution_some.first()

Row(country='FR', city='Ain', parameter='no2', value=7.9, utc_time='2021-10-06T09:00:00.000Z', local_time='2021-10-06T11:00:00+02:00', rolling_24h_avg=7.9)

In [49]:
from pyspark.sql.functions import to_timestamp

airpollution_some = airpollution_some.withColumn("utc_time", to_timestamp("utc_time", "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))

In [50]:
airpollution_some.printSchema()

root
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: double (nullable = true)
 |-- utc_time: timestamp (nullable = true)
 |-- local_time: string (nullable = true)
 |-- rolling_24h_avg: double (nullable = true)



In [73]:
first_table = airpollution_some.drop("value")

### Create id column for new data

In [74]:
table_name = "airpollution"
query = f"(SELECT * FROM {table_name} LIMIT 1) AS subquery"
df = spark.read.jdbc(url=url, table=query, properties=properties)
df.show()

+--------+--------------+-------+----+---------+----------+---------------+
|utc_time|measurement_id|country|city|parameter|local_time|rolling_24h_avg|
+--------+--------------+-------+----+---------+----------+---------------+
+--------+--------------+-------+----+---------+----------+---------------+



In [77]:
# Find the maximum ID in the DataFrame
max_id = df.selectExpr("MAX(measurement_id)").first()[0]
# Calculate the offset for generating new IDs
offset = max_id + 1 if max_id is not None else 1

In [78]:
from pyspark.sql.functions import monotonically_increasing_id, col, lit

# Generate new IDs with the offset
first_table = first_table.withColumn("measurement_id", monotonically_increasing_id() + lit(offset))

In [79]:
first_table.show(3)

+-------+----+---------+-------------------+--------------------+-----------------+--------------+
|country|city|parameter|           utc_time|          local_time|  rolling_24h_avg|measurement_id|
+-------+----+---------+-------------------+--------------------+-----------------+--------------+
|     FR| Ain|      no2|2021-10-06 09:00:00|2021-10-06T11:00:...|              7.9|             1|
|     FR| Ain|      no2|2021-10-06 09:00:00|2021-10-06T11:00:...|              9.4|             2|
|     FR| Ain|      no2|2021-10-06 10:00:00|2021-10-06T12:00:...|9.299999999999999|             3|
+-------+----+---------+-------------------+--------------------+-----------------+--------------+
only showing top 3 rows



### Insert the first table into TimescaleDB

In [80]:
mode = "append"
first_table.write.jdbc(url=url, table="airpollution", mode=mode, properties=properties)

### B. Compute Air Quality index for PM2.5 and PM10

In [81]:
from pyspark.sql.functions import udf, expr, lit, collect_list
from pyspark.sql.types import FloatType

In [82]:
pm25aqi_index = [{"aqi_low": 0, "aqi_high": 50, "breakpoint_low": 0, "breakpoint_high": 12},
                {"aqi_low": 51, "aqi_high": 100, "breakpoint_low": 12.1, "breakpoint_high": 35.4},
                {"aqi_low": 101, "aqi_high": 150, "breakpoint_low": 35.5, "breakpoint_high": 55.4},
                {"aqi_low": 151, "aqi_high": 200, "breakpoint_low": 55.5, "breakpoint_high": 150.4},
                {"aqi_low": 201, "aqi_high": 300, "breakpoint_low": 150.5, "breakpoint_high": 250.4},
                {"aqi_low": 301, "aqi_high": 400, "breakpoint_low": 250.5, "breakpoint_high": 350.4},
                {"aqi_low": 401, "aqi_high": 500, "breakpoint_low": 350.5, "breakpoint_high": 500.4},
                {"aqi_low": 501, "aqi_high": 999, "breakpoint_low": 500.5, "breakpoint_high": 99999.9},
                ]

In [83]:
pm10aqi_index = [{"aqi_low": 0, "aqi_high": 50, "breakpoint_low": 0, "breakpoint_high": 12},
                {"aqi_low": 51, "aqi_high": 100, "breakpoint_low": 55.0, "breakpoint_high": 154.0},
                {"aqi_low": 101, "aqi_high": 150, "breakpoint_low": 155.0, "breakpoint_high": 254.0},
                {"aqi_low": 151, "aqi_high": 200, "breakpoint_low": 255.0, "breakpoint_high": 354.0},
                {"aqi_low": 201, "aqi_high": 300, "breakpoint_low": 355.0, "breakpoint_high": 424.0},
                {"aqi_low": 301, "aqi_high": 400, "breakpoint_low": 425.0, "breakpoint_high": 504.0},
                {"aqi_low": 401, "aqi_high": 500, "breakpoint_low": 505.0, "breakpoint_high": 604.0},
                {"aqi_low": 501, "aqi_high": 999, "breakpoint_low": 604.0, "breakpoint_high": 99999.9},
                ]

In [84]:
def compute_aqi(pm24h: float, pm_type: str):
    aqi_high = None
    if pm_type=="pm10":
        score_table = pm10aqi_index
    elif pm_type=="pm25":
        score_table = pm25aqi_index
    else:
        print(f"There is no type like {type}")
        return None
    for max_and_min in score_table:
        if pm24h > max_and_min["breakpoint_low"] and pm24h < max_and_min["breakpoint_high"]:
            aqi_high = max_and_min["aqi_high"]
            aqi_low = max_and_min["aqi_low"]
            breakpoint_high = max_and_min["breakpoint_high"]
            breakpoint_low = max_and_min["breakpoint_low"]
            break
    if aqi_high is None:
        return None
    aqi = (aqi_high - aqi_low)/(breakpoint_high - breakpoint_low) * (pm24h - breakpoint_low) + aqi_low
    return aqi

In [85]:
# List of columns to drop
columns_to_drop = ["country", "city", "utc_time", "local_time"]

# Drop the specified columns
pm_airpollution = first_table.drop(*columns_to_drop)

In [86]:
condition1 = (first_table["parameter"] == "pm25")
condition2 = (first_table["parameter"] == "pm10")
combined_condition = condition1 | condition2

In [87]:
pm_airpollution = pm_airpollution.filter(combined_condition)

In [88]:
custom_udf = udf(compute_aqi, FloatType())

# Apply the UDF to a DataFrame column
pm_airpollution = pm_airpollution.withColumn("air_quality_index", custom_udf(pm_airpollution["rolling_24h_avg"], pm_airpollution["parameter"]))

In [89]:
# List of columns to drop
columns_to_drop = ["parameter", "rolling_24h_avg"]

# Drop the specified columns
second_table = pm_airpollution.drop(*columns_to_drop)

In [90]:
second_table.show(2)

+--------------+-----------------+
|measurement_id|air_quality_index|
+--------------+-----------------+
|            45|        35.833332|
|            46|        30.833334|
+--------------+-----------------+
only showing top 2 rows



### Insert the second table into TimescaleDB

In [91]:
mode = "append"
second_table.write.jdbc(url=url, table="air_quality_score", mode=mode, properties=properties)